In [ ]:
<a href="https://colab.research.google.com/github/AiratGaliev/llama-cpp-server-colab/blob/main/llama_cpp_server_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Start llama-cpp-server

#@markdown If unsure about the branch, write "main" or leave it blank.
%cd /content
!apt-get -y install -qq aria2
!curl -s https://ngrok-agent.s3.amazonaws.com/ngrok.asc | tee /etc/apt/trusted.gpg.d/ngrok.asc >/dev/null && echo "deb https://ngrok-agent.s3.amazonaws.com buster main" | tee /etc/apt/sources.list.d/ngrok.list
!apt update
!apt list --upgradable
!apt dist-upgrade
!apt install ngrok
!CUDACXX=/usr/local/cuda/bin/nvcc CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python[server]==0.2.38

json_cfg = {
    "host": "localhost",
    "port": 1234,
    "models": [
        {
            "model": "https://huggingface.co/TheBloke/dolphin-2.6-mistral-7B-dpo-laser-GGUF",
            "model_alias": "dolphin",
            "chat_format": "chatml"
        },
        {
            "model": "https://huggingface.co/TheBloke/openchat-3.5-0106-GGUF",
            "model_alias": "openchat",
            "chat_format": "openchat"
        },
        {
            "model": "https://huggingface.co/TheBloke/xDAN-L1-Chat-RL-v1-GGUF",
            "model_alias": "xdan",
            "chat_format": "chatml"
        },
        {
            "model": "https://huggingface.co/TheBloke/OpenHermes-2.5-Mistral-7B-16k-GGUF",
            "model_alias": "openhermes",
            "chat_format": "chatml"
        }
    ]
}

# Parameters
ngrok_auth_token = "" #@param {type:"string"}
model_url = "TheBloke/dolphin-2.6-mistral-7B-dpo-laser-GGUF" #@param {type:"string"}
model_alias = "dolphin-2.6-mistral-7b-dpo-laser" #@param {type:"string"}
chat_format = "chatml" #@param {type:"string"}
n_gpu_layers = 50 #@param {type:"integer"}
offload_kqv = True #@param {type:"boolean"}
n_threads = 12 #@param {type:"integer"}
n_batch = 2048 #@param {type:"integer"}
n_ctx = 16384 #@param {type:"integer"}
cache = False #@param {type:"boolean"}
quant_method = "Q8_0" # @param ["Q4_K_M", "Q5_K_M", "Q6_K", "Q8_0"]

models_dir = "/content/models"

for models in json_cfg["models"]:
    models["model"]: str = models_dir + models["model"].split('/')[-1].lower().rstrip(
        "-gguf") + f".{quant_method}.gguf"
    models["n_gpu_layers"]: int = n_gpu_layers
    models["offload_kqv"]: bool = offload_kqv
    models["n_threads"]: int = n_threads
    models["n_batch"]: int = n_batch
    models["n_ctx"]: int = n_ctx
    models["cache"]: bool = cache

model_url = model_url.strip()
model = ""
if model_url != "":
    if not model_url.startswith('http'):
        model_url = f"https://huggingface.co/{model_url}/resolve/main/{model_alias}.Q8_0.gguf"

# Download models
download_cmd = f"aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {model_url} -d {models_dir} -o {model_alias}.Q8_0.gguf"
print(download_cmd)
!$download_cmd

# Start server
model = f"{models_dir}/{model_alias}.Q8_0.gguf"
host = json_cfg["host"]
port = json_cfg["port"]

!python3 -m llama_cpp.server --host {host} --port {port} --model /content/models/dolphin-2.6-mistral-7b-dpo-laser.Q8_0.gguf --model_alias {model_alias} --chat_format {chat_format} --n_gpu_layers {n_gpu_layers} --offload_kqv {offload_kqv} --n_threads {n_threads} --n_batch {n_batch} --n_ctx {n_ctx} --cache {cache} > server.log 2>&1 &

# Start ngrok tunnel
# add secret variable NGROK_AUTHTOKEN with Authtoken value from https://dashboard.ngrok.com/get-started/your-authtoken
!ngrok config add-authtoken {ngrok_auth_token}
!ngrok http --domain=wasp-immortal-factually.ngrok-free.app {port}

In [ ]:
!pkill ngrok